# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store Quickstart** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Iris Classification</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/iris_tutorial.ipynb)

**Note**: you may get an error when installing hopsworks on Colab, and it is safe to ignore it.

This is the quickstart tutorial about Hopsworks Feature Store. Here you will work with data related to iris flower classification.


## 🗒️ This notebook is divided into next sections:
1. Import libraries and connect to Hopsworks Feature Store
2. Load the Iris flower dataset
3. Create a feature group and upload to the feature store
4. Create a feature view from the feature group
5. Create a training dataset
6. Train a model using XGBoost
7. Save the trained model to Hopsworks
8. Launch a serving instance.
9. Model deployment in Hopsworks
10. Send a prediction request to the served model
11. Try out your Model Interactively with a Gradio UI 

![tutorial-flow](../images/03_model.png)

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
!pip install -U xgboost --quiet

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import joblib
import json
import time

import pandas as pd
from sklearn import preprocessing

import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import f1_score, confusion_matrix

from matplotlib import pyplot
import seaborn as sns

import warnings

# Mute warnings
warnings.filterwarnings("ignore")

---

## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

In [ ]:
iris_df = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/iris.csv")
iris_df.head()

In [ ]:
iris_df.info()

---

## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

Lets save a feature group (hive table) called `iris` that contains the iris features and the corresponding numeric labels.

In [ ]:
import hopsworks


project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
iris_fg = fs.get_or_create_feature_group(name="iris",
                                         version=1,
                                         primary_key=[
                                             "sepal_length", "sepal_width",
                                             "petal_length", "petal_width"
                                         ],
                                         description="Iris flower dataset")

iris_fg.insert(iris_df, wait=True)

---

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

Feature views are used to read features for training and inference.
If the feature view already exists, get it. If not, create the feature view.

In [ ]:
query = iris_fg.select_all()

feature_view = fs.get_or_create_feature_view(name="iris",
                                             version=1,
                                             description="Read from Iris flower dataset",
                                             labels=["variety"],
                                             query=query)

In [ ]:
query_df = query.read()

In [ ]:
query_df

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using `fs.create_train_validation_test_split()` method.

* `X_train` is the train set features
* `X_val` is the validation set features
* `X_test` is the test set features
* `Y_train` is the train set labels
* `Y_val` is the validation set labels
* `Y_test` is the test set labels

In [ ]:
td_version, td_job = feature_view.create_train_validation_test_split(
    description='iris tutorial',
    data_format='csv',
    validation_size=0.2,
    test_size=0.1,
    write_options={'wait_for_job': True},
    coalesce = True,
)

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = feature_view.get_train_validation_test_split(td_version)

---

## <span style="color:#ff5f27;"> 🧬 Modeling</span>

Train the XGBoost Classifier model.

In [ ]:
# Use a label encoder to map the categorical labels to numbers.
le = preprocessing.LabelEncoder()
y_train_encoded=le.fit_transform(y_train['variety'])
y_test_encoded = le.transform(y_test['variety'])

In [ ]:
classifier = xgb.XGBClassifier()

classifier.fit(X_train.values, y_train_encoded)

### <span style='color:#ff5f27'> 📐 Model Validation

In [ ]:
plot_importance(classifier, max_num_features=10, importance_type='weight')

In [ ]:
y_pred = classifier.predict(X_test)

f1 = f1_score(y_test_encoded, y_pred, average="macro")

metrics = {
    "f1_score" : f1
}
print(metrics)

In [ ]:
y_test_encoded

In [ ]:
y_pred

In [ ]:
results = confusion_matrix(y_test_encoded, y_pred)
print(results)

In [ ]:
df_cm = pd.DataFrame(results, ['True Setosa', 'True Versicolor', 'True Virginica'],
                     ['Pred Setosa', 'Pred Versicolor', 'Pred Virginica'])

cm = sns.heatmap(df_cm, annot=True)

fig = cm.get_figure()
fig.show()

---

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/3.0/user_guides/mlops/registry/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train.values)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

## <span style="color:#ff5f27;">📝 Register model</span>

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Save the following objects as pickle files locally to a directory that will be uploaded later to the model registry:

 * the model object, `classifier` saved as `iris_xgboost_model.pkl`
 * the label encoder object, `le` saved as `iris_encoder.pkl`, so that we can reconstruct categorical names 
    from the encoded predictions (numbers) 
    
The model input schema is the same set of features as in the `X_train` DataFrame.

The model output schema is the same label as in the `y_train_encoded` array.

Finally, lazily create the model that will be register, including all files (artifacts) in the directory (containing the pickled label encoder object and the pickled model object), the model's input/output schema, and a sample input row (`input_example`). The model registry is the `mr` object, and for our Scikit-Learn model, we create a model of type Python with `mr.python.create_model()`. For TensorFlow, there is `mr.tensorflow.create_model()`.

In [ ]:
import joblib
import os


# The 'iris_model' directory will be saved to the model registry
model_dir="iris_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(classifier, model_dir + '/xgboost_iris_model.pkl')
joblib.dump(le, model_dir + '/iris_encoder.pkl')

# lets also save confusion matrix for this model version
fig.savefig(model_dir + "/confusion_matrix.png") 

In [ ]:
mr = project.get_model_registry()

iris_model = mr.python.create_model(
    name="xgboost_iris_model", 
    metrics=metrics,
    model_schema=model_schema,
    input_example=X_train.sample(), 
    description="Iris Flower Predictor")

iris_model.save(model_dir)

---

## <a class="anchor" id="1.5_bullet" style="color:#ff5f27"> 🚀 Model Deployment</a>

Models can be served via KFServing or "default" serving, which means a Docker container exposing a Flask server. For KFServing models, or models written in Tensorflow, you do not need to write a prediction file (see the section below). However, for sklearn models using default serving, you do need to proceed to write a prediction file.

In order to use KFServing, you must have Kubernetes installed and enabled on your cluster.

### <span style="color:#ff5f27;">📎 Predictor script for Python models</span>


Scikit-learn and XGBoost models are deployed as Python models, in which case you need to provide a **Predict** class that implements the **predict** method. The **predict()** method invokes the model on the inputs and returns the prediction as a list.

The **init()** method is run when the predictor is loaded into memory, loading the model from the local directory it is materialized to, *ARTIFACT_FILES_PATH*.

The directive `%%writefile` writes out the cell before to the given Python file. We will use the `predict_example.py` file to create a deployment for our model. 

In [ ]:
%%writefile predict_example.py

import os
import joblib


class Predict(object):
    
    def __init__(self):
        # NOTE: env var ARTIFACT_FILES_PATH has the local path to the model artifact files      
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/xgboost_iris_model.pkl")

    def predict(self, inputs):
        """ Serves a prediction request from a trained model"""
        return self.model.predict(inputs).tolist()

In [ ]:
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

In [ ]:
deployment = iris_model.deploy(name="irisdeployed",
                               script_file=predictor_script_path)

In [ ]:
print("Deployment: " + deployment.name)
deployment.describe()

In [ ]:
print("Deployment is warming up...")
time.sleep(45)

The deployment has now been registered. However, to start it you need to run:

In [ ]:
deployment.start()

In [ ]:
deployment.get_state().describe()

In [ ]:
# to troubleshoot you can use `get_logs()` method
deployment.get_logs(component='predictor')

### <span style='color:#ff5f27'>🔮 Predicting using deployment</span>

In [ ]:
res = deployment.predict(inputs=iris_model.input_example)
# or deployment.predict({ "instances": [iris_model.input_example] })

print(le.inverse_transform([res["predictions"][0]]))

---

## <span style="color:#ff5f27;"> 👾 Try out your Model Interactively </span> 


We will build a user interface with Gradio to allow you to enter the 4 feature values (sepal length/width and petal length/width), producing a prediction of the type of iris flower.

First, we have to install the gradio library.

In [ ]:
!pip install gradio --quiet
!pip install typing-extensions==4.3.0

### Run Gradio

Start the Gradio UI. Users enter the 4 feature values and a prediction is returned. We use the label encoder object to transform the number returned to the categorical value (stringified name of the Iris Flower).

In [ ]:
import gradio as gr


def iris(sl, sw, pl, pw):
    list_inputs = []
    list_inputs.append(sl)
    list_inputs.append(sw)
    list_inputs.append(pl)
    list_inputs.append(pw)
    res = deployment.predict(inputs=list_inputs)
    # Convert the numerical representation of the label back to it's original iris flower name.
    return le.inverse_transform([res["predictions"][0]])[0]

demo = gr.Interface(
    fn=iris,
    title="Iris Flower Predictive Analytics",
    description="Experiment with sepal/petal lengths/widths to predict which flower it is.",
    allow_flagging="never",
    inputs=[
        gr.inputs.Number(default=1.0, label="sepal length (cm)"),
        gr.inputs.Number(default=1.0, label="sepal width (cm)"),
        gr.inputs.Number(default=1.0, label="petal length (cm)"),
        gr.inputs.Number(default=1.0, label="petal width (cm)"),
        ],
    outputs="text")

demo.launch(share=True)